In [ ]:
! pip install "numpy<=2.1" # will be requiring as librosa is not compatible with numpy 2.2

In [3]:
from pyannote.audio import Pipeline
from transformers import pipeline
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
import librosa
import os
from indicnlp.normalize.indic_normalize import DevanagariNormalizer

d:\final karya\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
hf_token = os.environ.get('HF_TOKEN')

In [5]:
diarization_pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=hf_token )
audio_file= "outputs/videoplayback.wav"
diarization = diarization_pipeline(audio_file)

INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
C:\Users\anipr\AppData\Local\Programs\Python\Python312\Lib\inspect.py:1001: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):
d:\final karya\myenv\Lib\site-packages\pyannote\audio\models\blocks\pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\ReduceOps.cpp:1831.)
  std = sequences.std(dim=-1, correction=1)


In [6]:
whisper_asr = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-large-v2",
)
indic_factory = IndicNormalizerFactory()
normalizer = DevanagariNormalizer(
    lang='hi',
    remove_nuktas=False,
    nasals_mode='do_nothing',
    do_normalize_chandras=False,
    do_normalize_vowel_ending=False
)

Device set to use cpu


In [ ]:
transcript = []
for segment, _, speaker in diarization.itertracks(yield_label=True):
    
            # Load audio segment
            audio, sr = librosa.load(
                "outputs/videoplayback.wav",
                sr=16000,
                offset=segment.start,
                duration=segment.duration,
                mono=True
            )

            # Perform ASR on the segment
            result = whisper_asr(
                audio,
                return_timestamps=True
            )

            # Extract text from the result
            text = result["text"]

            # Normalize text if needed
            indic_factory = IndicNormalizerFactory()
            normalizer = indic_factory.get_normalizer("hi")
            text = normalizer.normalize(text)
            
            # Append segment information to the transcript
            transcript.append({
                "speaker": speaker,
                "start": float(segment.start),
                "end": float(segment.end),
                "text": text
            })


Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


In [ ]:
transcript